# Libraries

In [1]:
import os
nnn = 1
os.environ["OMP_NUM_THREADS"] = str(nnn) # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = str(nnn) # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = str(nnn) # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = str(nnn) # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = str(nnn)  # export NUMEXPR_NUM_THREADS=1

In [2]:
from TELF.factorization.HNMFk import HNMFk
import numpy as np
import os

In [3]:
import sys; sys.path.append(os.path.join("..", "..", "scripts"))
from generate_X import gen_data,gen_data_sparse

In [4]:
import TELF
TELF.__version__

'0.0.29'

# Generate Synthetic Data

In [5]:
Xsp = gen_data_sparse(shape=[500, 500], density=0.01)["X"]
X = gen_data(R=4, shape=[500, 500])["X"]

/home/maksim/repositories/telf_internal/examples/HNMFk/../../scripts/generate_X.py:60: UserWarning: Kruskal's theorem probably won't apply, may not have a unique nCPD.
  warnings.warn("Kruskal's theorem probably won't apply, may not have a unique nCPD.")


# Settings

In [6]:
Ks = np.arange(1, 10, 1)
perts = 2
iters = 1000
eps = 0.015
init = "nnsvd"
save_path = "HNMFk_results_path"
name = "example_HNMFk3"

In [7]:
nmfk_params = {
    "n_perturbs":perts,
    "n_iters":iters,
    "epsilon":eps,
    "n_jobs":2,
    "init":init, 
    "use_gpu":False,
    "save_path":save_path, 
    "predict_k_method":"sill",
    "verbose":False,
    "nmf_verbose":False,
    "transpose":False,
    "sill_thresh":0.8,
    "pruned":True,
    'nmf_method':'nmf_fro_mu',
    "calculate_error":False,
    "use_consensus_stopping":0,
    "calculate_pac":False,
    "consensus_mat":False,
    "perturb_type":"uniform",
    "perturb_multiprocessing":False,
    "perturb_verbose":False,
    "simple_plot":True,
    "k_search_method":"bst_post",
    "H_sill_thresh":0.1
}

In [8]:
hnmfk_params = {
    # we can specify nmfk parameters for each depth, or use same for all depth
    # below will use the same nmfk parameters for all depths
    # when using for each depth, append to the list 
    # for example, [nmfk_params0, nmfk_params1, nmfk_params2] for depth of 2
    "nmfk_params": [nmfk_params], 
    # where to perform clustering, can be W or H
    # if W, row of X should be samples
    # if H, columns of X should be samples
    "cluster_on":"H",
    # how deep to go in each topic after root node
    # if -1, it goes until samples cannot be seperated further
    "depth":3,
    # stopping criteria for num of samples
    "sample_thresh":100,
    # if K2=True, decomposition is done only for k=2 instead of 
    # finding and predicting the number of stable latent features
    "K2":False,
    # after first nmfk, when selecting Ks search range, minimum k to start
    "Ks_deep_min":1,
    # After first nmfk, when selecting Ks search range, maximum k to try.
    # When None, maximum k will be same as k selected for parent node.
    "Ks_deep_max": 20,
    # after first nmfk, when selecting Ks search range, k step size
    "Ks_deep_step":1,
    # where to save
    "experiment_name":name,
    # when True, names the nodes randomly.
    # When False, uses k index for ancestry naming
    "random_identifiers":False
}

In [9]:
model = HNMFk(**hnmfk_params)
model.fit(X, Ks, from_checkpoint=True, save_checkpoint=True)

/home/maksim/repositories/telf_internal/TELF/factorization/HNMFk.py:933: UserWarning: No checkpoint file found!
  warnings.warn("No checkpoint file found!")
/home/maksim/repositories/telf_internal/TELF/factorization/NMFk.py:699: UserWarning: predict_k_method is defaulted to WH_sill!
  warnings.warn("predict_k_method is defaulted to WH_sill!")
/home/maksim/miniconda3/envs/TELF/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/maksim/miniconda3/envs/TELF/lib/python3.11/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Done


{'time': 6.808085203170776}

# Example Traverse Graph

 Below functions can be utilized to walk the graph:

 ```python
model.traverse_nodes()
model.go_to_root()
model.get_node()
model.go_to_parent()
model.go_to_children(idx:int)
model.go_to_node(node_name:str)
 ```

 We can reset the iterator to go back to the root node as shown below:

In [10]:
node = model.go_to_root()
node["node_name"]

'*'

 HNMFk class includes a iterator that enables walking the graph nodes. Current node the iterator is at can be obtained as shown below (always starts at root node):

In [11]:
node = model.get_node()
node.keys()

dict_keys(['node_name', 'depth', 'W', 'H', 'k', 'parent_topic', 'parent_node_k', 'parent_node_name', 'child_node_names', 'original_indices', 'num_samples', 'leaf', 'user_node_data', 'cluster_indices_in_parent', 'node_save_path', 'parent_node_factors_path', 'parent_node_save_path', 'exception', 'signature', 'probabilities', 'centroids', 'factors_path'])

 We can also see the name of the node:

In [12]:
node["node_name"]

'*'

 And we can see the child nodes:

In [13]:
node["child_node_names"]

['*_0', '*_1', '*_2', '*_3']

 We can go to the child node specified with an index. For example, to go to the first child, we index at 0. When we go to the child node, it will return the child node and set the iterator to the child node.

In [14]:
node = model.go_to_children(1)
node["node_name"]

'*_1'

In [15]:
node.keys()

dict_keys(['node_name', 'depth', 'W', 'H', 'k', 'parent_topic', 'parent_node_k', 'parent_node_name', 'child_node_names', 'original_indices', 'num_samples', 'leaf', 'user_node_data', 'cluster_indices_in_parent', 'node_save_path', 'parent_node_factors_path', 'parent_node_save_path', 'exception', 'signature', 'probabilities', 'centroids', 'factors_path'])

In [16]:
node["node_save_path"]

'example_HNMFk3/depth_1/*_1/node_*_1.p'

In [17]:
node["parent_node_save_path"]

'example_HNMFk3/depth_0/*/node_*.p'

Go to a specific node

In [18]:
node = model.go_to_node("*_3")
node["child_node_names"]

['*_3_0', '*_3_1', '*_3_2', '*_3_3']

 Take a look at the parent node, which should be the root:

In [19]:
node["parent_node_name"]

'*'

get_node() always returns the node we are currently at:

In [20]:
node = model.get_node()
node["node_name"]

'*_3'

Go back to parent:

In [21]:
node = model.go_to_parent()
node["node_name"]

'*'

 From each node, we can get the samples that was clustered in the node:

In [22]:
node["original_indices"][:5]

array([0, 1, 2, 3, 4])

We can also look at which of the parent H clustering indices resulted in the samples in current cluster.

In [23]:
node = model.go_to_node("*_3_0")
print("Original indices from X=", node["original_indices"])
print("H clustering indices from parent topic=", node["cluster_indices_in_parent"])

Original indices from X= [ 32  38  55  59 140 176 178 204 274 299 305 338 345 413 415 477 485 489
 496]
H clustering indices from parent topic= [ 27  32  47  50 121 152 153 174 237 260 265 296 303 361 363 415 422 424
 430]


Root is empty because it does not have a parent:

In [24]:
node = model.go_to_root()
print("H clustering indices from parent topic=", node["cluster_indices_in_parent"])

H clustering indices from parent topic= []


We can also check if a given node in the graph a leaf node

In [25]:
node["leaf"]

False

 Finally, we can obtain all the nodes using the following method. Note that while other other node iterator options above are online, meaning each node is loaded into memory one at a time, the following traversal will load all nodes into the memory:

In [26]:
all_nodes = model.traverse_nodes()
len(all_nodes)

13

In [27]:
indices = []
leaf_nodes = []
for node in all_nodes:
    if node["leaf"]:
        indices += list(node["original_indices"])
        leaf_nodes.append(node)
indices.sort()
assert all(indices == np.arange(0, X.shape[1], 1))

In [28]:
len(leaf_nodes)

10

In [29]:
leaf_nodes[0]

{'node_name': '*_0',
 'depth': 1,
 'W': None,
 'H': None,
 'k': None,
 'parent_topic': 0,
 'parent_node_k': 4,
 'parent_node_name': '*',
 'child_node_names': [],
 'original_indices': array([ 20,  39,  77, 112, 208, 240, 249, 263, 370, 372, 406, 432, 439,
        442, 466, 487]),
 'num_samples': 16,
 'leaf': True,
 'user_node_data': {},
 'cluster_indices_in_parent': array([ 20,  39,  77, 112, 208, 240, 249, 263, 370, 372, 406, 432, 439,
        442, 466, 487]),
 'node_save_path': 'example_HNMFk3/depth_1/*_0/node_*_0.p',
 'parent_node_factors_path': 'example_HNMFk3/depth_0/*/NMFk-*_2perts_1000iters_0.015eps_nnsvd-init/WH_k=4.npz',
 'parent_node_save_path': 'example_HNMFk3/depth_0/*/node_*.p',
 'exception': False,
 'signature': array([1.70325395e-03, 4.14496893e-03, 4.47777240e-03, 8.82128661e-04,
        7.72891450e-04, 1.72561698e-03, 1.96789461e-03, 5.06446406e-04,
        2.36227410e-03, 1.63572864e-03, 4.09639487e-03, 1.86754309e-03,
        3.07867932e-03, 3.74930399e-03, 1.43890583

In [30]:
leaf_nodes[0]["centroids"].shape

(4, 16)

In [31]:
leaf_nodes[0]["signature"].shape

(500,)

In [32]:
leaf_nodes[0]["probabilities"].shape

(16,)

In [33]:
model.go_to_node("*_2")

{'node_name': '*_2',
 'depth': 1,
 'W': None,
 'H': None,
 'k': None,
 'parent_topic': 2,
 'parent_node_k': 4,
 'parent_node_name': '*',
 'child_node_names': [],
 'original_indices': array([ 34,  53,  56,  90, 129, 132, 151, 183, 195, 221, 275, 346, 347,
        417, 478, 491]),
 'num_samples': 16,
 'leaf': True,
 'user_node_data': {},
 'cluster_indices_in_parent': array([ 34,  53,  56,  90, 129, 132, 151, 183, 195, 221, 275, 346, 347,
        417, 478, 491]),
 'node_save_path': 'example_HNMFk3/depth_1/*_2/node_*_2.p',
 'parent_node_factors_path': 'example_HNMFk3/depth_0/*/NMFk-*_2perts_1000iters_0.015eps_nnsvd-init/WH_k=4.npz',
 'parent_node_save_path': 'example_HNMFk3/depth_0/*/node_*.p',
 'exception': False,
 'signature': array([4.52539418e-03, 9.23468149e-04, 2.49131210e-03, 3.62107535e-08,
        2.32034340e-03, 3.71468296e-08, 3.12231877e-03, 3.67488120e-08,
        5.48158120e-03, 8.14323605e-04, 2.49806838e-03, 3.01112956e-03,
        1.21856248e-03, 3.21191899e-03, 9.15026700

# Loading checkpoint to have access to the graph iterator and the node objects

In [34]:
model = HNMFk(experiment_name="example_HNMFk3")
model.load_model()

Loading saved object state from checkpoint...


In [35]:
model.node_save_paths

{'*': 'example_HNMFk3/depth_0/*/node_*.p',
 '*_0': 'example_HNMFk3/depth_1/*_0/node_*_0.p',
 '*_1': 'example_HNMFk3/depth_1/*_1/node_*_1.p',
 '*_2': 'example_HNMFk3/depth_1/*_2/node_*_2.p',
 '*_3': 'example_HNMFk3/depth_1/*_3/node_*_3.p',
 '*_3_0': 'example_HNMFk3/depth_2/*_3_0/node_*_3_0.p',
 '*_3_1': 'example_HNMFk3/depth_2/*_3_1/node_*_3_1.p',
 '*_3_2': 'example_HNMFk3/depth_2/*_3_2/node_*_3_2.p',
 '*_3_3': 'example_HNMFk3/depth_2/*_3_3/node_*_3_3.p',
 '*_3_3_0': 'example_HNMFk3/depth_3/*_3_3_0/node_*_3_3_0.p',
 '*_3_3_1': 'example_HNMFk3/depth_3/*_3_3_1/node_*_3_3_1.p',
 '*_3_3_2': 'example_HNMFk3/depth_3/*_3_3_2/node_*_3_3_2.p',
 '*_3_3_3': 'example_HNMFk3/depth_3/*_3_3_3/node_*_3_3_3.p'}

In [36]:
model.go_to_node("*_0")

{'node_name': '*_0',
 'depth': 1,
 'W': None,
 'H': None,
 'k': None,
 'parent_topic': 0,
 'parent_node_k': 4,
 'parent_node_name': '*',
 'child_node_names': [],
 'original_indices': array([ 20,  39,  77, 112, 208, 240, 249, 263, 370, 372, 406, 432, 439,
        442, 466, 487]),
 'num_samples': 16,
 'leaf': True,
 'user_node_data': {},
 'cluster_indices_in_parent': array([ 20,  39,  77, 112, 208, 240, 249, 263, 370, 372, 406, 432, 439,
        442, 466, 487]),
 'node_save_path': 'example_HNMFk3/depth_1/*_0/node_*_0.p',
 'parent_node_factors_path': 'example_HNMFk3/depth_0/*/NMFk-*_2perts_1000iters_0.015eps_nnsvd-init/WH_k=4.npz',
 'parent_node_save_path': 'example_HNMFk3/depth_0/*/node_*.p',
 'exception': False,
 'signature': array([1.70325395e-03, 4.14496893e-03, 4.47777240e-03, 8.82128661e-04,
        7.72891450e-04, 1.72561698e-03, 1.96789461e-03, 5.06446406e-04,
        2.36227410e-03, 1.63572864e-03, 4.09639487e-03, 1.86754309e-03,
        3.07867932e-03, 3.74930399e-03, 1.43890583

In [37]:
node = model.go_to_node(list(model.node_save_paths.keys())[0])
node["node_name"], node.keys()

('*',
 dict_keys(['node_name', 'depth', 'W', 'H', 'k', 'parent_topic', 'parent_node_k', 'parent_node_name', 'child_node_names', 'original_indices', 'num_samples', 'leaf', 'user_node_data', 'cluster_indices_in_parent', 'node_save_path', 'parent_node_factors_path', 'parent_node_save_path', 'exception', 'signature', 'probabilities', 'centroids', 'factors_path']))

Look at the nodes that are not completed yet

In [38]:
model.target_jobs.keys()

dict_keys([])

# Understanding the Naming Convention
 When ```"random_identifiers":False```, model uses k index for ancestry naming 

 - ```*```: This means the node is root
 - ```*_2```: This means it is at depth 1. The parent of the node came from cluster *, i.e. it is the root. The current node came from cluster 2 of the parent.
 - ```*_2_0```: This is at depth 2. Root node came from cluster *, then at root we went to cluster 2, and within this decomposition we took cluster 0 which to get to the corrent node.

(*) --> 0,1,(2)
(2) --> (0)